In [4]:
import gymnasium as gym


#定义环境
class MyWrapper(gym.Wrapper):

    def __init__(self):
        env = gym.make('CartPole-v1')
        super().__init__(env)
        self.env = env

    def reset(self,**kwargs):

        return self.env.reset(**kwargs)

    def step(self, action):
        state, reward, terminated, truncated, info = self.env.step(action)
        return state, reward, terminated, truncated, info


env = MyWrapper()

env.reset()

(array([-0.03873307, -0.00695048, -0.00101888, -0.01556431], dtype=float32),
 {})

In [ ]:
import torch
#对rewards进行decay后求和
decay = torch.arange(10)
#这里gamma写1.0意味着不decay
decay = 1.0**decay
decay

In [40]:
#认识游戏环境
def test_env(env):
    print('env.observation_space=', env.observation_space)
    print('env.action_space=', env.action_space)

    state = env.reset()
    action =env.action_space.sample()
    next_state, reward, terminated, truncated, info = env.step(action)

    print('state=', state)
    print('action=', action)
    print('next_state=', next_state)
    print('reward=', reward)
    print('done=', terminated) 


test_env(env)

env.observation_space= Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
env.action_space= Discrete(2)
state= (array([-0.03956558,  0.01132763, -0.01758445,  0.04715787], dtype=float32), {})
action= 1
next_state= [-0.03933902  0.20669726 -0.01664129 -0.25102082]
reward= 1.0
done= False


In [52]:
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import PPO

#初始化模型
model = PPO(
    policy='MlpPolicy',
    env=make_vec_env(MyWrapper, n_envs=4),  #创建N个环境用于训练
    n_steps=1024,
    batch_size=64,
    n_epochs=4,
    gamma=0.999,
    gae_lambda=0.98,
    ent_coef=0.01,
    verbose=0)

model

In [53]:
from stable_baselines3.common.evaluation import evaluate_policy

#测试
evaluate_policy(model, env, n_eval_episodes=10, deterministic=False)

(16.1, 7.077428911688199)

In [54]:
#训练
model.learn(total_timesteps=2_000, progress_bar=True)
model.save('models/ppo-LunarLander-v2')

In [55]:
model = PPO.load('models/ppo-LunarLander-v2')

evaluate_policy(model, env, n_eval_episodes=10, deterministic=False)

(28.9, 15.946472964263917)

In [7]:
from huggingface_sb3 import load_from_hub

#!pip install huggingface-sb3

#加载其他训练好的模型
#https://huggingface.co/models?library=stable-baselines3
model = PPO.load(
    load_from_hub('araffin/ppo-LunarLander-v2', 'ppo-LunarLander-v2.zip'),
    custom_objects={
        'learning_rate': 0.0,
        'lr_schedule': lambda _: 0.0,
        'clip_range': lambda _: 0.0,
    },
    print_system_info=True,
)

evaluate_policy(model, env, n_eval_episodes=10, deterministic=False)

Downloading:   0%|          | 0.00/144k [00:00<?, ?B/s]

== CURRENT SYSTEM INFO ==
- OS: Linux-5.15.0-3.60.5.1.el9uek.x86_64-x86_64-with-glibc2.34 # 2 SMP Wed Oct 19 20:27:31 PDT 2022
- Python: 3.9.15
- Stable-Baselines3: 1.8.0a1
- PyTorch: 1.13.0+cpu
- GPU Enabled: False
- Numpy: 1.23.5
- Gym: 0.26.2

== SAVED MODEL SYSTEM INFO ==
OS: Linux-5.13.0-40-generic-x86_64-with-debian-bullseye-sid #45~20.04.1-Ubuntu SMP Mon Apr 4 09:38:31 UTC 2022
Python: 3.7.10
Stable-Baselines3: 1.5.1a5
PyTorch: 1.11.0
GPU Enabled: False
Numpy: 1.21.2
Gym: 0.21.0



/root/anaconda3/envs/pt39/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(250.9974542026721, 86.61020518339575)